# Find the county to each station

### Takes the unfiltered CSV and filters it into desired columns


In [1]:
import pandas as pd
import os
from geopy.geocoders import Nominatim

Reads it and writes a new csv file

In [3]:
# column 'G' is stationid, 'L' is lat, 'M' is long 
unprocessed = pd.read_csv('TMAS2015_august_laye.csv')
unprocessed.head()
new = unprocessed.filter(items = ['G','L','M','F'])

for index,row in new.iterrows():
    new.at[index,'G'] = '0'*(6-len(row['G'])) + row['G']

#changes the names of the columns 
new = new.rename(columns={ 'G': 'Station_Id', 'L': 'Latitude', 'M': 'Longitude', 'F': 'State_Code' })
new['County'] = ''
new.to_csv(path_or_buf='station_location.csv',index = False)


        G        L         M   F
0   50223  21.9348 -159.4835  15
1   50015  21.9681 -159.3931  15
2     300  22.2096 -159.4134  15
3     400  20.9130 -156.6892  15
4  340005  20.9235 -156.5026  15


,G,L,M,F
0,050223,21.9348,-159.4835,15
1,050015,21.9681,-159.3931,15
2,000300,22.2096,-159.4134,15
3,000400,20.9130,-156.6892,15
4,340005,20.9235,-156.5026,15


## Reverse address to find county using GeoPy


Creates the instance of the Address finder



In [4]:
increment = 1000
rows = 0

adress_reverser = Nominatim(user_agent="adress_reverser")

#gets rid of the county and ' in the county string
def county_formatter(county_str):
    if county_str != None:
        return county_str.replace(' County','').replace('ʻ','')
    return ''

#gets the county using the latitude and longitude of each row
def change_row(row):
    coord = (row['Latitude'], row['Longitude'])
    location = adress_reverser.reverse(coord, exactly_one = True)
    address = location.raw['address']
    county = address.get('county')
    return county_formatter(county)

Takes only a thousand rows of the station locations at a time

In [ ]:
stations = pd.read_csv('station_location.csv', nrows = increment, skiprows=rows+1, names = ['Station_Id','Latitude','Longitude', 'State_Code','County'], dtype = {'Latitude' : str, 'Longitude' : str, 'Station_Id' : str, 'State_Code' : int, 'County' : str})
rows += increment

stations['County'] = stations.apply(change_row,axis = 1)
stations.to_csv('county_added.csv',index = False,mode = 'a')

### Fix up the code, fill in the missing gaps by manually inputing the latitude and longitude in a map to find the county
- final csv should look like *station_county_key.csv*
